# Feature Engineering
In this notebook, we will engineer count based features based on existing features. We note that the number of features which can be engineered through count is very large

In [1]:
# Ignore deprecated warning
import warnings
warnings.filterwarnings("ignore")

# Data manipulation
%matplotlib inline
import pandas as pd
import numpy as np
import time

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set font scale and style
plt.rcParams.update({'font.size': 15})

In [2]:
df = pd.read_csv('../data/clean_data.csv') #  Load  data
print('Data size',df.shape)
df.head()

Data size (3817613, 23)


,c_year,c_mnth,c_wday,c_hour,class,c_vehs,c_conf,c_rcfg,c_wthr,c_rsur,...,v_type,v_year,p_id,p_sex,p_age,p_psn,p_isev,p_safe,p_user,c_case
0,1999,January,Monday,9.0,0,2.0,Right turn,At an intersection,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1992.0,1.0,F,33.0,Driver,Injury,Safety device used,Motor Vehicle Driver,2890
1,1999,January,Monday,9.0,0,2.0,Right turn,At an intersection,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1992.0,1.0,F,70.0,Driver,No Injury,Safety device used,Motor Vehicle Driver,2890
2,1999,January,Monday,20.0,0,1.0,Ran off left shoulder,Intersection with parking lot entrance,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1988.0,1.0,F,38.0,Driver,Injury,Safety device used,Motor Vehicle Driver,4332
3,1999,January,Monday,5.0,0,2.0,Hit a moving object,At an intersection,Raining,Wet,...,Other trucks and vans,1995.0,1.0,M,34.0,Driver,No Injury,Safety device used,Motor Vehicle Driver,5053
4,1999,January,Monday,5.0,0,2.0,Hit a moving object,At an intersection,Raining,Wet,...,Other trucks and vans,1995.0,2.0,M,30.0,"Front row, right outboard",No Injury,Safety device used,Motor Vehicle Passenger,5053


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3817613 entries, 0 to 3817612
Data columns (total 23 columns):
 #   Column  Dtype  
---  ------  -----  
 0   c_year  int64  
 1   c_mnth  object 
 2   c_wday  object 
 3   c_hour  float64
 4   class   int64  
 5   c_vehs  float64
 6   c_conf  object 
 7   c_rcfg  object 
 8   c_wthr  object 
 9   c_rsur  object 
 10  c_raln  object 
 11  c_traf  object 
 12  v_id    float64
 13  v_type  object 
 14  v_year  float64
 15  p_id    float64
 16  p_sex   object 
 17  p_age   float64
 18  p_psn   object 
 19  p_isev  object 
 20  p_safe  object 
 21  p_user  object 
 22  c_case  int64  
dtypes: float64(6), int64(3), object(14)
memory usage: 669.9+ MB


# 1. Day, month, and year counts

In [4]:
# Day count
df_day = pd.DataFrame(df.c_wday.value_counts())
df_day.reset_index(level=0,inplace=True)
df_day.rename(columns = {'c_wday':'wday_ct',  'index':'c_wday'},inplace=True)

# Month count
df_mnth = pd.DataFrame(df.c_mnth.value_counts())
df_mnth.reset_index(level=0,inplace=True)
df_mnth.rename(columns = {'c_mnth':'mnth_ct',  'index':'c_mnth'},inplace=True)

# Year count
df_year = pd.DataFrame(df.c_year.value_counts())
df_year.reset_index(level=0,inplace=True)
df_year.rename(columns = {'c_year':'year_ct',  'index':'c_year'},inplace=True)

In [5]:
# left join to the main dataframe
df = df.merge(df_day, how = 'left', on = 'c_wday') 
df = df.merge(df_mnth, how = 'left', on = 'c_mnth')
df = df.merge(df_year, how = 'left', on = 'c_year') 

In [6]:
# Average features
df['avg_day_per_mnth'] =  df.wday_ct/df.mnth_ct
df['avg_mnth_per_year'] =  df.mnth_ct/df.year_ct

# 2. Hour, weather, configurations counts

In [7]:
# Hour count
df_hour = pd.DataFrame(df.c_hour.value_counts())
df_hour.reset_index(level=0,inplace=True)
df_hour.rename(columns = {'c_hour':'hour_ct',  'index':'c_hour'},inplace=True)

# Weather count
df_wthr = pd.DataFrame(df.c_wthr.value_counts())
df_wthr.reset_index(level=0,inplace=True)
df_wthr.rename(columns = {'c_wthr':'wthr_ct',  'index':'c_wthr'},inplace=True)

# Configuration count
df_conf = pd.DataFrame(df.c_conf.value_counts())
df_conf.reset_index(level=0,inplace=True)
df_conf.rename(columns = {'c_conf':'conf_ct',  'index':'c_conf'},inplace=True)

# Road way configuration count
df_rcfg = pd.DataFrame(df.c_rcfg.value_counts())
df_rcfg.reset_index(level=0,inplace=True)
df_rcfg.rename(columns = {'c_rcfg':'rcfg_ct',  'index':'c_rcfg'},inplace=True)

In [8]:
# left join to the main dataframe
df = df.merge(df_hour, how = 'left', on = 'c_hour')
df = df.merge(df_wthr, how = 'left', on = 'c_wthr') 
df = df.merge(df_conf, how = 'left', on = 'c_conf') 
df = df.merge(df_rcfg, how = 'left', on = 'c_rcfg') 

In [9]:
# Average features
df['avg_hr_per_day'] =  df.hour_ct/df.wday_ct
df['avg_wthr_per_day'] =  df.wthr_ct/df.wday_ct
df['avg_conf_per_day'] =  df.conf_ct/df.wday_ct
df['avg_rcfg_per_day'] =  df.rcfg_ct/df.wday_ct

# 3. Surface, alignment and position counts

In [10]:
# Road surface count
df_rsur = pd.DataFrame(df.c_rsur.value_counts())
df_rsur.reset_index(level=0,inplace=True)
df_rsur.rename(columns = {'c_rsur':'rsur_ct',  'index':'c_rsur'},inplace=True)

# Road alignment count
df_raln = pd.DataFrame(df.c_raln.value_counts())
df_raln.reset_index(level=0,inplace=True)
df_raln.rename(columns = {'c_raln':'raln_ct',  'index':'c_raln'},inplace=True)

# Person position count
df_psn = pd.DataFrame(df.p_psn.value_counts())
df_psn.reset_index(level=0,inplace=True)
df_psn.rename(columns = {'p_psn':'psn_ct',  'index':'p_psn'},inplace=True)

In [11]:
# left join to the main dataframe
df = df.merge(df_rsur, how = 'left', on = 'c_rsur')
df = df.merge(df_raln, how = 'left', on = 'c_raln') 
df = df.merge(df_psn, how = 'left', on = 'p_psn') 

In [12]:
# Average features
df['avg_rsur_per_day'] =  df.rsur_ct/df.wday_ct
df['avg_raln_per_day'] =  df.raln_ct/df.wday_ct
df['avg_psn_per_day'] =  df.psn_ct/df.wday_ct

# 4. Treatment, vehicle types, traffic control counts

In [13]:
# Treatment required count
df_isev = pd.DataFrame(df.p_isev.value_counts())
df_isev.reset_index(level=0,inplace=True)
df_isev.rename(columns = {'p_isev':'isev_ct',  'index':'p_isev'},inplace=True)

# Vechicle type count
df_vtype = pd.DataFrame(df.v_type.value_counts())
df_vtype.reset_index(level=0,inplace=True)
df_vtype.rename(columns = {'v_type':'vtype_ct',  'index':'v_type'},inplace=True)

# Traffic control type
df_traf = pd.DataFrame(df.c_traf.value_counts())
df_traf.reset_index(level=0,inplace=True)
df_traf.rename(columns = {'c_traf':'traf_ct',  'index':'c_traf'},inplace=True)

In [14]:
# left join to the main dataframe
df = df.merge(df_isev, how = 'left', on = 'p_isev')
df = df.merge(df_vtype, how = 'left', on = 'v_type') 
df = df.merge(df_traf, how = 'left', on = 'c_traf') 

In [15]:
# Average features
df['avg_isev_per_day'] =  df.isev_ct/df.wday_ct
df['avg_vtype_per_day'] =  df.vtype_ct/df.wday_ct
df['avg_traf_per_day'] =  df.traf_ct/df.wday_ct

# Scatter plot

In [ ]:
vars_ = ['avg_isev_per_day', 'isev_ct', 'class', 'traf_ct']
sns.pairplot(df, vars = vars_, hue = 'class')
plt.show()

# Save as csv

In [ ]:
df.to_csv('../data/feat_engr_data.csv', index= False)

# Fisher Score

In [36]:
def fisher_score(df, col1, col2):
    """
    Calculate the Fisher score for variable selection

    Parameters
    ---------
    df: Pandas Dataframe
    col1: class variable column name
    col2: variable column name to calculate fisher score
    
    Returns
    --------
    Fisher score
    """
    pvt = pd.crosstab(df[col2], df[col1])

    mean = pvt[0].mean() - pvt[1].mean()
    variance = (pvt[0].var())**2 + (pvt[1].var())**2
    score = np.abs(mean)/np.sqrt(variance)
    print("Fisher score for {} is {}".format(str(col2), np.round(score,5)))

In [37]:
cols = list(df.drop('class', axis =1).select_dtypes('number'))
for col in cols:
    fisher_score(df, 'class', col)

Fisher score for c_year is 0.00034
Fisher score for c_hour is 2e-05
Fisher score for c_vehs is 0.0
Fisher score for v_id is 0.0
Fisher score for v_year is 1e-05
Fisher score for p_id is 0.0
Fisher score for p_age is 4e-05
Fisher score for c_case is 0.93347
Fisher score for wday_ct is 0.00016
Fisher score for mnth_ct is 0.0003
Fisher score for year_ct is 0.00034
Fisher score for avg_day_per_mnth is 0.00107
Fisher score for avg_mnth_per_year is 0.00232
Fisher score for hour_ct is 2e-05
Fisher score for wthr_ct is 0.0
Fisher score for conf_ct is 0.0
Fisher score for rcfg_ct is 0.0
Fisher score for avg_hr_per_day is 0.0001
Fisher score for avg_wthr_per_day is 0.0
Fisher score for avg_conf_per_day is 1e-05
Fisher score for avg_rcfg_per_day is 1e-05
Fisher score for rsur_ct is 0.0
Fisher score for raln_ct is 0.0
Fisher score for psn_ct is 0.0
Fisher score for avg_rsur_per_day is 0.0
Fisher score for avg_raln_per_day is 0.0
Fisher score for avg_psn_per_day is 0.0
Fisher score for isev_ct is 0